In [0]:
%run ./encryption_utils

In [0]:
from pyspark.sql.functions import (
    col, to_timestamp, year, month, abs as spark_abs, max as spark_max, min as spark_min,
    count, avg, datediff, explode, sequence, row_number, expr, lit, to_date, broadcast
)
from pyspark.sql.window import Window
from pyspark.sql.types import DoubleType, FloatType

encryptor = PIIEncryptor()

expedia_pii_columns = [
    "user_id",
    "user_location_country",
    "user_location_region",
    "user_location_city",
    "orig_destination_distance"
]
hotel_weather_pii_columns = [
    "address",
    "name"
]
# read silver data
expedia_df = spark.table("silver.expedia_processed")
hotel_weather_df = spark.table("silver.hotel_weather_processed")

#decrypt
expedia_df = df = encryptor.decrypt_dataframe(expedia_df, expedia_pii_columns)
hotel_weather_df = df = encryptor.decrypt_dataframe(hotel_weather_df, hotel_weather_pii_columns)


In [0]:

# Convert temperature columns to double
hotel_weather_df = hotel_weather_df.withColumn("avg_tmpr_c", col("avg_tmpr_c").cast(DoubleType())) \
                                   .withColumn("avg_tmpr_f", col("avg_tmpr_f").cast(DoubleType()))

# Cast Expedia check-in/out to timestamp
expedia_df = expedia_df.withColumn("srch_ci", to_timestamp(col("srch_ci"))) \
                       .withColumn("srch_co", to_timestamp(col("srch_co")))

# Top 10 Hotels with Max Temp Difference by Month
temp_stats_df = hotel_weather_df.withColumn("year", year(col("wthr_date"))) \
                                .withColumn("month", month(col("wthr_date"))) \
                                .groupBy("id", "year", "month") \
                                .agg(
                                    spark_max(col("avg_tmpr_c")).alias("max_temp"),
                                    spark_min(col("avg_tmpr_c")).alias("min_temp")
                                ) \
                                .withColumn("temp_difference", spark_abs(col("max_temp") - col("min_temp")))

window_temp = Window.partitionBy("year", "month").orderBy(col("temp_difference").desc())
top_10_temp_diff = temp_stats_df.withColumn("rank", row_number().over(window_temp)) \
                                .filter(col("rank") <= 10) \
                                .drop("rank") \
                                .orderBy("year", "month", col("temp_difference").desc())

top_10_temp_diff.write.format("delta").mode("overwrite").option("overwriteSchema", "true") \
    .saveAsTable("gold.top_10_temp_diff_monthly")



In [0]:
display(top_10_temp_diff)

id,year,month,max_temp,min_temp,temp_difference
1571958030336,2016,10,10.8,-7.7,18.5
77309411328,2016,10,19.2,1.0,18.2
695784701957,2016,10,15.2,-1.1,16.3
369367187456,2016,10,21.1,4.9,16.200000000000003
77309411335,2016,10,20.7,5.8,14.899999999999999
1520418422787,2016,10,20.7,5.8,14.899999999999999
1013612281860,2016,10,19.2,4.5,14.7
137438953473,2016,10,22.1,7.7,14.400000000000002
1108101562369,2016,10,16.3,2.1,14.200000000000001
335007449088,2016,10,17.3,3.8,13.5


In [0]:
# 1. Filter and select relevant columns
visits_df = expedia_df.filter(
    col("hotel_id").isNotNull() & col("srch_ci").isNotNull() & col("srch_co").isNotNull() & (to_date(col("srch_co")) >= to_date(col("srch_ci")))
).select(
    "hotel_id",
    col("srch_ci").alias("check_in"),
    col("srch_co").alias("check_out")
)

# 2. Explode stays into monthly visits
visits_month_df = visits_df.withColumn(
    "month_date",
    explode(
        sequence(
            to_date(col("check_in")),    # Start from check-in
            to_date(col("check_out")),   # End at check-out
            expr("interval 1 month")    # Step by 1 month
        )
    )
).withColumn("year", year(col("month_date"))) \
 .withColumn("month", month(col("month_date")))

# 3. Count visits per hotel per month
hotel_visits_df = visits_month_df.groupBy("hotel_id", "year", "month") \
                                .agg(count("*").alias("visit_count"))

# 4. Window to get top 10 busiest hotels per month
window_visits = Window.partitionBy("year", "month").orderBy(col("visit_count").desc())

top_10_busiest = hotel_visits_df.withColumn("rank", row_number().over(window_visits)) \
                                .filter(col("rank") <= 10) \
                                .drop("rank") \
                                .orderBy("year", "month", col("visit_count").desc())

# 5. Write to Gold table
top_10_busiest.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("gold.top_10_busiest_hotels_monthly")

In [0]:
display(top_10_busiest)

hotel_id,year,month,visit_count
1864015806473,2016,10,405
1958505086979,2016,10,396
936302870530,2016,10,395
1151051235330,2016,10,393
575525617665,2016,10,391
695784701953,2016,10,391
103079215105,2016,10,390
206158430209,2016,10,389
2920577761284,2016,10,389
2568390443017,2016,10,389


In [0]:
# 1. Filter extended stays (>7 days)
extended_stays_df = expedia_df.filter(
    datediff(col("srch_co"), col("srch_ci")) > 7
).select(
    "hotel_id",
    col("srch_ci").alias("check_in"),
    col("srch_co").alias("check_out")
)

# 2. Convert temperature to float
hotel_weather_df = hotel_weather_df.withColumn("avg_tmpr_c", col("avg_tmpr_c").cast(FloatType()))

# 3. Filter weather for relevant stays
weather_filtered_df = hotel_weather_df.alias("weather").join(
    extended_stays_df.alias("stay"),
    (col("weather.id") == col("stay.hotel_id")) &
    (col("weather.wthr_date").between(col("stay.check_in"), col("stay.check_out"))),
    "inner"
).select(
    col("stay.hotel_id"),
    col("stay.check_in"),
    col("stay.check_out"),
    col("weather.wthr_date"),
    col("weather.avg_tmpr_c")
)

# 4. Define windows to get first and last day temps
window_first = Window.partitionBy("hotel_id", "check_in", "check_out").orderBy("wthr_date")
window_last  = Window.partitionBy("hotel_id", "check_in", "check_out").orderBy(col("wthr_date").desc())

weather_trend_extended = weather_filtered_df.withColumn("first_day_temp", row_number().over(window_first)) \
    .withColumn("last_day_temp", row_number().over(window_last)) \
    .where((col("first_day_temp") == 1) | (col("last_day_temp") == 1)) \
    .groupBy("hotel_id", "check_in", "check_out") \
    .agg(
        avg("avg_tmpr_c").alias("avg_temp_during_stay"),
        spark_min("avg_tmpr_c").alias("first_day_temp"),  # first day's temperature
        spark_max("avg_tmpr_c").alias("last_day_temp")    # last day's temperature
    ).withColumn(
        "weather_trend", col("last_day_temp") - col("first_day_temp")
    ).withColumn(
        "stay_duration", datediff(col("check_out"), col("check_in"))
    )

# 5. Write to Gold
weather_trend_extended.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("gold.weather_trend_extended_stay")

In [0]:
display(weather_trend_extended)

hotel_id,check_in,check_out,avg_temp_during_stay,first_day_temp,last_day_temp,weather_trend,stay_duration
0,2017-08-01T00:00:00Z,2017-08-12T00:00:00Z,23.100000381469727,23.0,23.2,0.20000076,11
0,2017-08-04T00:00:00Z,2017-08-14T00:00:00Z,21.899999618530273,20.8,23.0,2.2000008,10
0,2017-08-07T00:00:00Z,2017-08-17T00:00:00Z,23.0,23.0,23.0,0.0,10
0,2017-09-15T00:00:00Z,2017-09-29T00:00:00Z,17.75,16.9,18.6,1.7000008,14
0,2017-09-16T00:00:00Z,2017-10-14T00:00:00Z,17.75,16.9,18.6,1.7000008,28
0,2017-09-19T00:00:00Z,2017-10-03T00:00:00Z,17.75,16.9,18.6,1.7000008,14
0,2017-09-24T00:00:00Z,2017-10-17T00:00:00Z,17.75,16.9,18.6,1.7000008,23
0,2017-09-26T00:00:00Z,2017-10-05T00:00:00Z,17.75,16.9,18.6,1.7000008,9
1,2016-10-04T00:00:00Z,2016-10-15T00:00:00Z,12.050000190734863,7.5,16.6,9.1,11
1,2016-10-06T00:00:00Z,2016-10-15T00:00:00Z,12.050000190734863,7.5,16.6,9.1,9
